# Initializing

In [1]:
#Active environment should be aws_env
!conda info | grep 'active env'

     active environment : aws_env
    active env location : /home/hassan101/anaconda3/envs/aws_env


In [2]:
#Get AWS credentials from environment
import os
aws_akid = os.environ['AWS_KID']
aws_sak = os.environ['AWS_AK']

In [3]:
#Importing libraries
import boto3
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Getting processed data from S3 bucket

In [4]:
#Authentication
res_s3 = boto3.resource('s3', aws_access_key_id=aws_akid, aws_secret_access_key= aws_sak)
client_s3 = boto3.client('s3', aws_access_key_id=aws_akid, aws_secret_access_key= aws_sak) 

In [5]:
# List all objects in bucket
bucket_name = 'reg-dataset-processed'

response = client_s3.list_objects_v2(Bucket=bucket_name)
for obj in response['Contents']:
    print(obj)

{'Key': 'lifexp-processed.csv', 'LastModified': datetime.datetime(2023, 2, 9, 6, 39, 41, tzinfo=tzutc()), 'ETag': '"a0d4c354e748204bcba7e714cc7f8c88"', 'Size': 109036, 'StorageClass': 'STANDARD'}


In [7]:
response = client_s3.get_object(Bucket=bucket_name, Key="lifexp-processed.csv")

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    df = pd.read_csv(response.get("Body"))
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200


In [9]:
df.head(20)

,HIV/AIDS,Income composition of resources,Adult mortality,Under-five deaths,Thinness 5-9 years,Status,Polio,Total expenditure
0,0.1,0.479,263.0,83,17.3,Developing,6.0,8.16
1,0.1,0.476,271.0,86,17.5,Developing,58.0,8.18
2,0.1,0.470,268.0,89,17.7,Developing,62.0,8.13
3,0.1,0.463,272.0,93,18.0,Developing,67.0,8.52
4,0.1,0.454,275.0,97,18.2,Developing,68.0,7.87
5,0.1,0.448,279.0,102,18.4,Developing,66.0,9.20
6,0.1,0.434,281.0,106,18.7,Developing,63.0,9.42
7,0.1,0.433,287.0,110,18.9,Developing,64.0,8.33
8,0.1,0.415,295.0,113,19.1,Developing,63.0,6.73
9,0.1,0.405,295.0,116,19.3,Developing,58.0,7.43
